In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/random-data/innings-dataset.csv
/kaggle/input/ipl/deliveries.csv
/kaggle/input/ipl/matches.csv


In [3]:
match = pd.read_csv('/kaggle/input/ipl/matches.csv')
delivery = pd.read_csv('/kaggle/input/ipl/deliveries.csv')
innings = pd.read_csv('/kaggle/input/random-data/innings-dataset.csv')

In [4]:
innings.sample(5)

,mid,date,venue,bat_team,bowl_team,batsman,bowler,runs,wickets,overs,runs_last_5,wickets_last_5,striker,non-striker,total
73331,596,2017-05-01,Wankhede Stadium,Royal Challengers Bangalore,Mumbai Indians,Mandeep Singh,SL Malinga,16,0,1.5,16,0,14,2,162
39085,317,2013-04-06,Feroz Shah Kotla,Rajasthan Royals,Delhi Daredevils,R Dravid,S Nadeem,67,1,9.2,34,0,24,22,165
25469,206,2011-05-01,Sawai Mansingh Stadium,Pune Warriors,Rajasthan Royals,M Manhas,SR Watson,122,5,18.1,27,1,16,0,143
75019,609,2017-05-11,Wankhede Stadium,Kings XI Punjab,Mumbai Indians,SE Marsh,SL Malinga,171,2,14.4,46,1,66,16,230
19077,154,2010-04-11,Feroz Shah Kotla,Delhi Daredevils,Kings XI Punjab,A Mishra,PP Chawla,102,8,17.5,26,3,24,5,111


In [5]:
# 1. Venue avg score
# 2. Batting team avg score 
# 3. Bowling team avg run conceeded

In [6]:
delivery = delivery.merge(match,left_on='match_id',right_on='id')

In [7]:
delivery = delivery[delivery['inning']==1]

In [8]:
runs_per_stadium = delivery.groupby('venue').sum()['total_runs']

In [9]:
match_per_stadium = match['venue'].value_counts()

In [10]:
avg_score_stadium = runs_per_stadium/match_per_stadium

In [11]:
avg_score_stadium = avg_score_stadium.reset_index()

In [12]:
avg_score_stadium.rename(columns={'index':'venue',0:'avg_score_stadium'},inplace=True)

In [13]:
avg_score_stadium.head()

,venue,avg_score_stadium
0,Barabati Stadium,167.714286
1,Brabourne Stadium,180.363636
2,Buffalo Park,147.333333
3,De Beers Diamond Oval,158.000000
4,Dr DY Patil Sports Academy,146.647059


In [21]:
import pickle

In [22]:
pickle.dump(avg_score_stadium, open('venue_avg.pkl','wb'))

In [14]:
innings = innings.merge(avg_score_stadium,on='venue')

In [15]:
runs_avg_batting_team = innings.drop_duplicates(subset=['mid']).groupby('bat_team').mean()['total']

In [16]:
runs_avg_bowling_team = innings.drop_duplicates(subset=['mid']).groupby('bowl_team').mean()['total']

In [17]:
runs_avg_batting_team = runs_avg_batting_team.reset_index()
runs_avg_bowling_team = runs_avg_bowling_team.reset_index()

In [18]:
runs_avg_batting_team.head()

,bat_team,total
0,Chennai Super Kings,165.716216
1,Deccan Chargers,157.325581
2,Delhi Daredevils,155.172414
3,Gujarat Lions,161.928571
4,Kings XI Punjab,160.346667


In [24]:
pickle.dump(runs_avg_batting_team, open('batting_avg.pkl','wb'))

In [25]:
pickle.dump(runs_avg_bowling_team, open('bowling_avg.pkl','wb'))

In [19]:
runs_avg_batting_team.rename(columns={'index':'bat_team',0:'avg_runs_batting_team'},inplace=True)
runs_avg_bowling_team.rename(columns={'index':'bowl_team',0:'avg_runs_bowling_team'},inplace=True)

In [20]:
runs_avg_batting_team

,bowl_team,total
0,Chennai Super Kings,160.490566
1,Deccan Chargers,162.718750
2,Delhi Daredevils,162.783133
3,Gujarat Lions,173.750000
4,Kings XI Punjab,166.647059
5,Kochi Tuskers Kerala,142.833333
6,Kolkata Knight Riders,155.146667
7,Mumbai Indians,158.640000
8,Pune Warriors,157.280000
9,Rajasthan Royals,157.111111


In [21]:
innings = innings.merge(runs_avg_batting_team,on='bat_team')

In [22]:
innings = innings.merge(runs_avg_bowling_team,on='bowl_team')

In [23]:
innings.rename(columns={'total_y':'avg_batting_team_score','total':'avg_bowling_team_score'},inplace=True)

In [24]:
innings.rename(columns={'total_x':'total'},inplace=True)

In [52]:
innings.corr()['total']

mid                       0.096586
runs                      0.254498
wickets                  -0.337343
overs                     0.022282
runs_last_5               0.368253
wickets_last_5           -0.273475
striker                   0.381664
non-striker               0.293233
total                     1.000000
avg_score_stadium         0.262129
avg_batting_team_score    0.161425
avg_bowling_team_score    0.149077
Name: total, dtype: float64

In [25]:
innings.drop(columns=['mid','date','batsman','bowler','striker','non-striker'],axis=1,inplace=True)

In [26]:
innings.sample(5)

,venue,bat_team,bowl_team,runs,wickets,overs,runs_last_5,wickets_last_5,total,avg_score_stadium,avg_batting_team_score,avg_bowling_team_score
8766,M Chinnaswamy Stadium,Sunrisers Hyderabad,Royal Challengers Bangalore,124,4,16.6,35,2,155,167.893939,162.833333,161.911392
12715,Wankhede Stadium,Chennai Super Kings,Mumbai Indians,116,4,17.3,43,0,156,163.947368,165.716216,158.640000
15279,Wankhede Stadium,Rajasthan Royals,Mumbai Indians,34,0,5.2,29,0,189,163.947368,156.851852,158.640000
63354,Saurashtra Cricket Association Stadium,Royal Challengers Bangalore,Gujarat Lions,178,2,17.6,52,1,213,168.900000,166.088235,173.750000
57466,"MA Chidambaram Stadium, Chepauk",Mumbai Indians,Chennai Super Kings,28,2,4.5,28,2,148,166.020833,163.256098,160.490566


In [ ]:
innings.shape

In [57]:
innings['bat_team'].unique()

array(['Kolkata Knight Riders', 'Chennai Super Kings', 'Kings XI Punjab',
       'Rajasthan Royals', 'Delhi Daredevils', 'Mumbai Indians',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Deccan Chargers',
       'Sunrisers Hyderabad', 'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Royal Challengers Bangalore'],
      dtype=object)

In [58]:
obsolete_teams = ['Kochi Tuskers Kerala','Pune Warriors','Deccan Chargers','Rising Pune Supergiants','Gujarat Lions','Rising Pune Supergiant']

In [59]:
innings = innings[~(innings['bat_team'].isin(obsolete_teams)) & ~(innings['bowl_team'].isin(obsolete_teams))]

In [ ]:
innings.head()

In [ ]:
innings.shape

In [60]:
from sklearn.preprocessing import LabelEncoder
venue_encoder = LabelEncoder()
team_encoder = LabelEncoder()

In [61]:
innings['venue'] = venue_encoder.fit_transform(innings['venue'])
innings['bat_team'] = team_encoder.fit_transform(innings['bat_team'])
innings['bowl_team'] = team_encoder.transform(innings['bowl_team'])

In [ ]:
innings.sample(5)

In [62]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first')

In [63]:
X = innings.drop(['total','venue','bat_team','bowl_team'],axis=1)

In [64]:
X_trans = ohe.fit_transform(innings[['venue','bat_team','bowl_team']]).toarray()

In [ ]:
X_trans.shape

In [65]:
X = np.hstack((X,X_trans))

In [ ]:
X.shape

In [66]:
y = innings['total'].values

In [67]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [69]:
pickle.dump(scaler,open('scaler.pkl','wb'))

In [68]:
X = scaler.fit_transform(X)

In [ ]:
X[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape

In [ ]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [ ]:
import kerastuner as kt

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
def build_model(hp):
    
    model = Sequential()
    counter = 0
    
    for i in range(hp.Int('num_layers',min_value=1,max_value=10)):
        if counter == 0 :
            model.add(
                Dense(
                    units=hp.Int('unit' + str(i),min_value=8,max_value=256,step=8),
                    activation=hp.Choice('activation',values=['relu','tanh','sigmoid','selu','elu']),
                    input_dim=X_train.shape[1],
                    kernel_regularizer = regularizers.l2(hp.Choice('reg' + str(i),values=[1e-5,1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3]))
                )
            )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.8,0.9])))
        else:
            model.add(
                Dense(
                    units=hp.Int('unit' + str(i),min_value=8,max_value=256,step=8),
                    activation=hp.Choice('activation',values=['relu','tanh','sigmoid','selu','elu']),
                    kernel_regularizer = regularizers.l2(hp.Choice('reg' + str(i),values=[1e-5,1e-4,1e-3,1e-2,1e-1,1e0,1e1,1e2,1e3]))
                )
            )
            model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.8,0.9])))
        counter = counter + 1
    model.add(Dense(1,activation='linear'))
    model.compile(optimizer=hp.Choice('opti',['rmsprop','adam','sgd','adagrad','adadelta','nadam','adamax']),
                 loss = 'mse',
                 metrics=['mse'])
    return model  

In [ ]:
tuner = kt.RandomSearch(build_model,objective='mse',max_trials=3,directory='somedir1',project_name='ipl1')

In [ ]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

In [ ]:
model = tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

In [ ]:
model.fit(X_train,y_train,epochs=100,initial_epoch=5,validation_data=(X_test,y_test))

In [ ]:
history = model.history.history

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history['mse'])
plt.plot(history['val_mse'])

In [ ]:
model.evaluate(X_test,y_test)

In [ ]:
import pickle

In [ ]:
# encoders
# ohe
# model

#pickle.dump(model,open('model.pkl','wb'))
pickle.dump(ohe,open('ohe.pkl','wb'))
pickle.dump(team_encoder,open('team_encoder.pkl','wb'))
pickle.dump(venue_encoder,open('venue_encoder.pkl','wb'))

In [ ]:
model.save('model.h5')